In [1]:

import numpy as np
import scipy as sp
import h5py

import os

import SimData as sim_data
import Simulator as sim_system
import Optimization as opt
import SurrogateDEV2 as mDE

In [2]:


###* Input files and function

reactions_file = "reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"

In [3]:


def func_optimization(params):
    
    A_O, B_O, E_O, \
    A_CO, B_CO, E_CO = params
    
    nu_d_mod = lambda T: 1e15 * (A_O + B_O * np.exp(E_O/(const_dict['R'] * T)))
    nu_d_mod_2 = lambda T: 1e15 * (A_CO + B_CO * np.exp(E_CO/(const_dict['R'] * T)))
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod_2}}
    ]
    
    return dict_mod_vec

loss_function = lambda exp, teo: np.mean((np.reciprocal(exp)*(exp-teo))**2)

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)


  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [4]:

if __name__ == '__main__':
    optimizer = opt.Optimizer(sim, func_optimization, loss_function)

    global_bounds = [(1e-5, 1.0), (1e-8, 1.0), (5, 30), \
                    (1e-5, 1.0), (1e-8, 1.0), (5, 30)]

    config = {
        "bounds": global_bounds,
        "nb_calls": 3, # 5
        "de_maxiter": 10,
        "de_num_iterations": 2,
        "de_max_generations": 100,
        "top_k": 3,
        "local_attempts": 1,
        "epsilon_local": 1e-2
    }
    
    ##3 objectvive fucno
    
    def my_simulator(params):
        return optimizer.objective_function(params)
    
    
    opt_DE = mDE.SurrogateDE(my_simulator, global_bounds, popsize=5, max_gens=50,
                       surrogate_every=5, surrogate_min_points=50)
    
    best_x, best_f = opt_DE.run()
    print("Best solution:", best_x, best_f)
    
# differential_evolution step 50: f(x)= 0.041662857183863554


[Gen 1] i=0, f_trial=0.858648
[Gen 1] i=1, f_trial=0.858632
[Gen 1] i=2, f_trial=0.858644
[Gen 1] i=3, f_trial=0.858645
[Gen 1] i=4, f_trial=0.858642
[Gen 1] i=5, f_trial=0.858645
[Gen 1] i=6, f_trial=0.858643
[Gen 1] i=7, f_trial=0.858646
[Gen 1] i=8, f_trial=0.858635
[Gen 1] i=9, f_trial=0.858648
[Gen 1] i=10, f_trial=0.858643
[Gen 1] i=11, f_trial=0.858641
[Gen 1] i=12, f_trial=0.858636
[Gen 1] i=13, f_trial=0.858646
[Gen 1] i=14, f_trial=0.858645
[Gen 1] i=15, f_trial=0.858646
[Gen 1] i=16, f_trial=0.855943
[Gen 1] i=17, f_trial=0.855924
[Gen 1] i=18, f_trial=0.855916
[Gen 1] i=19, f_trial=0.855961
[Gen 1] i=20, f_trial=0.855920
[Gen 1] i=21, f_trial=0.860100
[Gen 1] i=22, f_trial=0.860099
[Gen 1] i=23, f_trial=0.860101
[Gen 1] i=24, f_trial=0.859999
[Gen 1] i=25, f_trial=0.860006
[Gen 1] i=26, f_trial=0.859990
[Gen 1] i=27, f_trial=0.860006
[Gen 1] i=28, f_trial=0.860006
[Gen 1] i=29, f_trial=0.860006
Gen   1: best_f = 0.533225
[Gen 2] i=0, f_trial=0.825163
[Gen 2] i=1, f_trial=0.

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Gen 10] injected 10 surrogate proposals
Gen  10: best_f = 0.168228
[Gen 11] i=0, f_trial=0.786034
[Gen 11] i=1, f_trial=0.774522
[Gen 11] i=2, f_trial=0.785790
[Gen 11] i=3, f_trial=0.781009
[Gen 11] i=4, f_trial=0.786233
[Gen 11] i=5, f_trial=0.781579
[Gen 11] i=6, f_trial=0.782743
[Gen 11] i=7, f_trial=0.786523
[Gen 11] i=8, f_trial=0.783579
[Gen 11] i=9, f_trial=0.787039
[Gen 11] i=10, f_trial=0.769753
[Gen 11] i=11, f_trial=0.780364
[Gen 11] i=12, f_trial=0.787637
[Gen 11] i=13, f_trial=0.784129
[Gen 11] i=14, f_trial=0.780196
[Gen 11] i=15, f_trial=0.782563
[Gen 11] i=16, f_trial=0.805560
[Gen 11] i=17, f_trial=0.804277
[Gen 11] i=18, f_trial=0.801333
[Gen 11] i=19, f_trial=0.808220
[Gen 11] i=20, f_trial=0.799809
[Gen 11] i=21, f_trial=0.833984
[Gen 11] i=22, f_trial=0.834040
[Gen 11] i=23, f_trial=0.832429
[Gen 11] i=24, f_trial=0.824964
[Gen 11] i=25, f_trial=0.825281
[Gen 11] i=26, f_trial=0.823429
[Gen 11] i=27, f_trial=0.824165
[Gen 11] i=28, f_trial=0.824363
[Gen 11] i=29,

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Gen 15] injected 10 surrogate proposals
Gen  15: best_f = 0.168228
[Gen 16] i=0, f_trial=0.795793
[Gen 16] i=1, f_trial=0.782865
[Gen 16] i=2, f_trial=0.795605
[Gen 16] i=3, f_trial=0.791996
[Gen 16] i=4, f_trial=0.784156
[Gen 16] i=5, f_trial=0.792421
[Gen 16] i=6, f_trial=0.788184
[Gen 16] i=7, f_trial=0.796159
[Gen 16] i=8, f_trial=0.793927
[Gen 16] i=9, f_trial=0.796559
[Gen 16] i=10, f_trial=0.788770
[Gen 16] i=11, f_trial=0.787974
[Gen 16] i=12, f_trial=0.797016
[Gen 16] i=13, f_trial=0.794346
[Gen 16] i=14, f_trial=0.794343
[Gen 16] i=15, f_trial=0.793165
[Gen 16] i=16, f_trial=0.813040
[Gen 16] i=17, f_trial=0.812079
[Gen 16] i=18, f_trial=0.809890
[Gen 16] i=19, f_trial=0.815047
[Gen 16] i=20, f_trial=0.808584
[Gen 16] i=21, f_trial=0.860057
[Gen 16] i=22, f_trial=0.860061
[Gen 16] i=23, f_trial=0.860061
[Gen 16] i=24, f_trial=0.255240
[Gen 16] i=25, f_trial=0.268965
[Gen 16] i=26, f_trial=4.149137
[Gen 16] i=27, f_trial=0.581142
[Gen 16] i=28, f_trial=0.401911
[Gen 16] i=29,

...

[Gen 40] injected 10 surrogate proposals
Gen  40: best_f = 0.168228
[Gen 41] i=0, f_trial=0.826517
[Gen 41] i=1, f_trial=0.825823
[Gen 41] i=2, f_trial=0.826465
[Gen 41] i=3, f_trial=0.825479
[Gen 41] i=4, f_trial=0.823615
[Gen 41] i=5, f_trial=0.825593
[Gen 41] i=6, f_trial=0.824650
[Gen 41] i=7, f_trial=0.826622
[Gen 41] i=8, f_trial=0.826001
[Gen 41] i=9, f_trial=0.826733
[Gen 41] i=10, f_trial=0.824475
[Gen 41] i=11, f_trial=0.823503
[Gen 41] i=12, f_trial=0.826864
[Gen 41] i=13, f_trial=0.826115
[Gen 41] i=14, f_trial=0.826180
[Gen 41] i=15, f_trial=0.825793
[Gen 41] i=16, f_trial=0.835975
[Gen 41] i=17, f_trial=0.835719
[Gen 41] i=18, f_trial=0.835154
[Gen 41] i=19, f_trial=0.836527
[Gen 41] i=20, f_trial=0.835649
[Gen 41] i=21, f_trial=0.856318
[Gen 41] i=22, f_trial=0.856328
[Gen 41] i=23, f_trial=0.856313
[Gen 41] i=24, f_trial=0.641333
[Gen 41] i=25, f_trial=0.651329
[Gen 41] i=26, f_trial=0.581115
[Gen 41] i=27, f_trial=0.612641
[Gen 41] i=28, f_trial=0.620202
[Gen 41] i=29,

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Gen 45] injected 10 surrogate proposals
Gen  45: best_f = 0.168228
[Gen 46] i=0, f_trial=0.860149
[Gen 46] i=1, f_trial=0.860149
[Gen 46] i=2, f_trial=0.860149
[Gen 46] i=3, f_trial=0.860149
[Gen 46] i=4, f_trial=0.860146
[Gen 46] i=5, f_trial=0.860149
[Gen 46] i=6, f_trial=0.860149
[Gen 46] i=7, f_trial=0.860148
[Gen 46] i=8, f_trial=0.860149
[Gen 46] i=9, f_trial=0.860149
[Gen 46] i=10, f_trial=0.860148
[Gen 46] i=11, f_trial=0.860149
[Gen 46] i=12, f_trial=0.860149
[Gen 46] i=13, f_trial=0.860149
[Gen 46] i=14, f_trial=0.860149
[Gen 46] i=15, f_trial=0.860149
[Gen 46] i=16, f_trial=0.860149
[Gen 46] i=17, f_trial=0.860149
[Gen 46] i=18, f_trial=0.860149
[Gen 46] i=19, f_trial=0.860149
[Gen 46] i=20, f_trial=0.860149
[Gen 46] i=21, f_trial=0.757149
[Gen 46] i=22, f_trial=0.751306
[Gen 46] i=23, f_trial=0.760149
[Gen 46] i=24, f_trial=0.456134
[Gen 46] i=25, f_trial=0.484565
[Gen 46] i=26, f_trial=0.270772
[Gen 46] i=27, f_trial=0.369752
[Gen 46] i=28, f_trial=0.393081
[Gen 46] i=29,

/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/miniconda3/envs/LLM/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Gen 50] injected 10 surrogate proposals
Gen  50: best_f = 0.151565
Best solution: [9.89648249e-03 1.95032785e-02 8.14289709e+00 1.00000000e-05
 5.30575798e-01 1.71766171e+01] 0.15156482936643215


In [5]:
# DE resul: [7.54561369e-02 3.30911944e-04 1.80716671e+01 6.55878140e-02
#  9.49664603e-01 5.15676107e+00] | 0.04027624574740497

# ####* Test default 

# import matplotlib.pyplot as plt

# params = np.array([0.01634, 1.67e-4, 19.75])



def do_plots(params):


    loss = optimizer.objective_function(params)
    print("PARAMS: ", params, "Loss: ", loss)

    frac_solutions_arr, gammas_results_arr, gammas_sum_arr, gamma_exp_data = optimizer.solve_simulations_updated(params)
    exp_data_arr = optimizer.exp_data_arr

    inv_Tnw_vec = np.array([300/vec['Tnw'] for vec in exp_data_arr])

    plt.figure
    plt.plot(inv_Tnw_vec, gamma_exp_data, '.', label="Exp")
    plt.plot(inv_Tnw_vec, gammas_sum_arr, '.', label="Sim")
    plt.legend()
    plt.xlabel(r"$300/T_{nw}$")
    plt.ylabel(r"$\gamma$")

    current_vec = np.array([vec['current'] for vec in exp_data_arr])
    Tw_vec = np.array([vec['Tw'] for vec in exp_data_arr])

    current_unique = np.unique(current_vec)
    Tw_unique = np.unique([vec['Tw'] for vec in exp_data_arr])


    A, B = np.meshgrid(current_unique, Tw_unique)
    combinations = np.stack([A.ravel(), B.ravel()], axis=1)

    for comb in combinations:
        current = comb[0]
        Tw = comb[1]

        idx_valid = np.logical_and(current_vec == current, Tw_vec == Tw)
        data_arr = exp_data_arr[idx_valid]
        
        pressure_vec = np.array([vec['pressure'] for vec in data_arr])
        gamma_exp = gamma_exp_data[idx_valid]
        gamma_sim = gammas_sum_arr[idx_valid]
        
        plt.figure()
        plt.plot(pressure_vec, gamma_exp*1e4, 's', label="Exp")
        plt.plot(pressure_vec, gamma_sim*1e4, 's', label="Sim")
        plt.title(f"I = {current} and Tw = {Tw}")
        plt.xlabel("Pressure")
        plt.ylabel(r"$\gamma$")
        plt.legend()
        plt.grid()
        plt.ylim(1, 14)
        plt.show()



In [6]:

params_1 = np.array([0.01634, 1.67e-4, 19.75])
do_plots(params_1)


ValueError: not enough values to unpack (expected 6, got 3)

In [ ]:





params_2 = np.array([2.39375716e-02, 7.11846527e-04, 1.64416626e+01])
do_plots(params_2)

In [ ]:

params_3 = np.array([8.32874763e-03, 8.32658918e-04, 1.61121534e+01])
do_plots(params_3)

In [ ]:

# def nu_d_func(T, params):
#     A, B, E = params
    
#     nu_d_mod = 1e15 * (A + B * np.exp(E * np.reciprocal(const_dict['R'] * T)))
    
#     return nu_d_mod


# T_space = np.linspace(-20+273.15, 50+273.15, 200)

# vec_1 = np.array([nu_d_func(T, params_1) for T in T_space])
# vec_2 = np.array([nu_d_func(T, params_2) for T in T_space])
# vec_3 = np.array([nu_d_func(T, params_3) for T in T_space])

# plt.plot(T_space, vec_1, label="Paper")
# plt.plot(T_space, vec_2, label="DE 1")
# plt.plot(T_space, vec_3, label="DE 2")
# plt.legend()
# plt.xlabel("Tw")
# plt.ylabel(r"$\nu_{d}$")